In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent, Crew, Task

In [5]:
# pip install --upgrade crewai crewai-tools

In [6]:
from dotenv import load_dotenv
import os

load_dotenv() 

openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
# Custom Tools

from crewai.tools import tool

class CalculatorTools():
    @tool("Make a calcualtion")
    def calculate(operation):
        """Useful to perform any mathematical calculations, 
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical 
        expression.`
        """
        return eval(operation)

## Creating Agents

In [9]:
market_researcher = Agent(
            role = "trend analyst",
            goal = "Analyze the latest trends in the travel industry and provide insights on popular destinations, activities, and travel preferences.",
            backstory = "You are a trend analyst specializing in the travel industry. Your task is to analyze the latest trends and provide insights on popular destinations, activities, and travel preferences. You will use various tools to gather information and perform calculations as needed.",
            tools=[search_tool, 
                   scrape_tool
            ],
            verbose=True,
        )


In [10]:
sentiment_analyst = Agent(
            role = "sentiment analyst",
            goal = "Analyze the sentiment of customer reviews and feedback to identify positive and negative aspects of travel experiences.",
            backstory = "You are a sentiment analyst specializing in the travel industry. Your task is to analyze customer reviews and feedback to identify positive and negative aspects of travel experiences. You will use various tools to gather information and perform sentiment analysis as needed.",
            tools=[search_tool],
            verbose=True,
        )

In [11]:
competitive_analyst = Agent(
            role = "competitive analyst",
            goal = "Analyze the competitive landscape of the travel industry, identifying key players, their strengths and weaknesses, and market positioning.",
            backstory = "You are a competitive analyst specializing in the travel industry. Your task is to analyze the competitive landscape, identifying key players, their strengths and weaknesses, and market positioning. You will use various tools to gather information and perform calculations as needed.",
            tools=[search_tool, 
                   scrape_tool,
                   CalculatorTools.calculate,
            ],
            verbose=True,
        )

In [12]:
strategy_planner = Agent(
    role = "strategy planner",
    goal = "Develop a comprehensive strategy for a travel company, including market entry, product development, and marketing strategies.",
    backstory = "You are a strategy planner specializing in the travel industry. Your task is to develop a comprehensive strategy for a travel company, including market entry, product development, and marketing strategies. You will use various tools to gather information and perform calculations as needed. And summarize the findings from the other agents.",
    tools=[search_tool, scrape_tool, CalculatorTools.calculate],
    verbose=True
)

## Creating Tasks

In [13]:
market_researcher_task = Task(
    description="Identify top 3 travel trends in the year {year} specifically for {target_audience}.",
    expected_output="A list of trends with brief explanations.",
    agent=market_researcher
)

In [14]:
sentiment_analyst_task = Task(
    description="Analyze reviews for {target_audience} in {region_focus} to find key likes/dislikes.",
    expected_output="Top sentiments from travel reviews.",
    agent=sentiment_analyst
)

In [15]:
competitive_analyst_task = Task(
    description="Compare travel platforms: {platforms_to_compare} in terms of features and market position.",
    expected_output="A comparison of their market standing and offerings.",
    agent=competitive_analyst
)

In [16]:
strategy_planner_task = Task(
    description="Using all findings, create a go-to-market strategy to {entry_goal} in {region_focus}.",
    expected_output="A concise 3-point strategic recommendation.",
    agent=strategy_planner
)

## Creating the Crew

In [17]:
# Define the crew with agents and tasks
market_trend_analyst_crew = Crew(
    agents=[market_researcher, sentiment_analyst, competitive_analyst, strategy_planner],
    tasks=[market_researcher_task, sentiment_analyst_task, competitive_analyst_task, strategy_planner_task],
    verbose=True
)

## Running the Crew

In [18]:
input_details = {
    "target_audience": "Gen Z travelers",
    "year": "2025",
    "platforms_to_compare": ["Expedia", "Booking.com", "Airbnb"],
    "entry_goal": "launch a competitive travel platform with innovative features",
    "region_focus": "North America"
}

In [19]:
result = market_trend_analyst_crew.kickoff(inputs=input_details)

# Agent: trend analyst
## Task: Identify top 3 travel trends in the year 2025 specifically for Gen Z travelers.


# Agent: trend analyst
## Thought: I need to gather the latest information on travel trends for Gen Z travelers in the year 2025.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Gen Z travel trends 2025\"}"
## Tool Output: 
{'searchParameters': {'q': 'Gen Z travel trends 2025', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Gen Z Travel Trends and Statistics in 2025 - Peek Pro', 'link': 'https://www.peekpro.com/blog/gen-z-travel-trends', 'snippet': "The data below highlights Gen Z's key travel behaviors, booking habits, and top destinations for 2025. 90% use social media for travel inspiration. 80% book trips using mobile apps and digital wallets. 56% prioritize sustainable and responsible travel choices .", 'position': 1}, {'title': '2025 Global Travel Trends Report - American Express', 'link': 'https://www.americanexpress

In [20]:
print(result)

1. **Develop a Platform Focused on Experience-Driven Travel**: Create a travel platform that prioritizes immersive, adventure-filled experiences catering to Gen Z's desire for unique and culturally-rich encounters. Incorporate user-generated content to showcase real-life experiences and promote activities such as cooking classes, adventure sports, and local craft workshops. This platform should enable easy customization of itineraries based on user interests and local offerings, enhancing the hyperlocal travel experience.

2. **Leverage Social Media and Influencer Partnerships**: Engage in aggressive digital marketing strategies that utilize platforms favored by Gen Z, such as Instagram and TikTok. Partner with relevant influencers who can authentically showcase travel experiences tailored to Gen Z's tastes. Utilize short-form video content to capture attention, and promote travel contests that encourage user participation and sharing, thereby enhancing brand visibility and engagement.

In [21]:
from IPython.display import Markdown, display

for task in getattr(result, "tasks_output", []):
    display(Markdown(task.raw))

1. **Immersive Experiences and Adventure Activities**: Gen Z travelers are increasingly seeking immersive and interactive experiences that they can share on social media. Activities like mini-golf, water sports, and other engaging adventures are becoming popular. These experiences not only provide excitement but also serve as shareable content for social media platforms, enhancing their travel experience through every activity.

2. **Hyperlocal Travel and Cultural Connections**: There is a significant shift towards hyperlocal travel, where Gen Z is more interested in culturally rich destinations that offer genuine connections with local traditions and communities. They prefer experiences that allow them to engage with local culture on a deeper level, rather than typical tourist attractions. This trend highlights the importance of personalized and meaningful travel experiences.

3. **Meaningful and Unique Take-Home Experiences**: Today's travelers prefer unique keepsakes that symbolize their experiences rather than traditional souvenirs. Gen Z is inclined towards hands-on activities that allow them to create something personal and lasting from their travels, such as cooking classes or crafting sessions that reflect local culture.
```

Based on the analysis of current trends and sources, here are the key likes and dislikes of Gen Z travelers in North America regarding their travel experiences:

### Likes:
1. **Immersive and Unique Experiences**: Gen Z shows a strong preference for immersive activities that allow them to engage deeply with local cultures. They prioritize activities like cooking classes, local crafts, and adventure sports over traditional sightseeing.
2. **Cultural Authenticity and Hyperlocal Travel**: They prefer destinations that provide cultural connections, favoring local traditions and authentic experiences over conventional tourist attractions.
3. **Social Media Sharing**: Experiences that can be easily shared on platforms like Instagram or TikTok are particularly appealing, driving their choices towards visually engaging activities.
4. **Sustainability and Ethics**: Gen Z travelers are more likely to choose brands and destinations that align with their values of sustainability and social responsibility, often willing to spend more if they believe their choices are environmentally friendly.
5. **Flexibility and Spontaneity**: Many Gen Z travelers enjoy the flexibility in their travel plans, often booking trips last minute and appreciating options that allow spontaneous changes to itineraries.

### Dislikes:
1. **Conventional and Overly Touristy Destinations**: Gen Z tends to shy away from mainstream tourist hotspots, preferring undiscovered places that offer unique experiences.
2. **High Costs and Lack of Value**: They are price-sensitive and seek good deals but are willing to pay a premium for experiences they perceive as valuable or unique. Poor value for money in standard tourist offerings can deter them.
3. **Bureaucratic Processes**: Many express frustration with complicated travel arrangements, such as visas or other travel restrictions that impede their ability to travel freely or spontaneously.
4. **Corporate and Sterile Environments**: Gen Z is generally averse to environments that feel commercial or impersonal, such as large chain hotels lacking unique features or character.

In summary, Gen Z travelers value experiences that are immersive, authentic, and easily shareable, while they tend to avoid traditional tourist activities that do not align with their individualistic values. Their preferences highlight a significant shift in travel behavior intended to seek depth in experiences rather than surface-level engagement.
```

### **Comparison of Travel Platforms: Expedia, Booking.com, and Airbnb**

---

#### **1. Expedia**
**Market Position:**  
Expedia Group is a leader in the online travel industry, offering a wide range of travel services, including booking accommodations, flights, car rentals, and vacation packages. It is focused on both B2C and B2B markets.

**Strengths:**
- **Comprehensive Offerings:** Expedia provides extensive travel services, allowing users to plan entire trips in one place—including flights, hotels, car rentals, and activities.
- **Brand Recognition:** As one of the oldest travel agencies online, it has strong brand recognition and consumer trust.
- **Budget-Friendly Options:** Regular promotions and discounts cater to price-sensitive travelers, appealing to Gen Z who seek good value.
  
**Weaknesses:**
- **Complex Interface:** Some users find the interface overwhelming with options, which could deter less experienced travelers.
- **Less Focus on Unique Experiences:** Traditionally, Expedia has been more focused on standard hotel offerings and may not cater specifically to Gen Z’s demand for immersive and unique experiences as effectively as its competitors.

---

#### **2. Booking.com**
**Market Position:**  
Booking.com is one of the most widely used platforms for travelers seeking accommodations globally. It has successfully expanded its offerings beyond just hotels to include apartments and unique stays.

**Strengths:**
- **Diverse Accommodation Types:** Offers hotels, homes, and unique properties, catering to a broad range of preferences.
- **User-Friendly Interface:** The highly intuitive interface eases the booking process, particularly attractive to tech-savvy Gen Z travelers.
- **Focus on Experiences:** Engaging with local experiences and adventures, Booking.com aligns well with Gen Z's preference for hyperlocal and immersive travel experiences, including cultural connections.

**Weaknesses:**
- **Market Saturation:** Faces strong competition from platforms like Airbnb and local OTA alternatives, which may impact growth.
- **Limited Control Over Unique Properties:** While diverse, the quality and uniqueness of offered properties may not always meet Gen Z’s high expectations for authenticity.

---

#### **3. Airbnb**
**Market Position:**  
Airbnb has carved a niche in the travel market by offering unique accommodations and local experiences not typically provided by hotels, targeting those who desire authentic cultural interactions.

**Strengths:**
- **Unique Inventory:** Airbnb provides a vast array of unique listings, including homes, treehouses, and even experiences catered to local culture, drawing in Gen Z's thirst for immersive adventures.
- **Local Host Interactions:** Enables connections with local hosts, fulfilling Gen Z's desire for meaningful cultural experiences.
- **Strong Brand Engagement:** Airbnb’s marketing strategies, including partnerships with cultural icons, resonate well with Gen Z, often generating viral social media content.

**Weaknesses:**
- **Quality Control Issues:** Varied quality across listings can lead to inconsistencies in the user experience, which can be frustrating for travelers seeking reliability.
- **Increased Regulatory Scrutiny:** Airbnb is facing growing regulations in many cities worldwide, affecting availability and pricing, which can dissuade potential users.

---

### **Conclusion**
In summary, travelers—especially Gen Z—are focused on unique and immersive experiences that foster cultural connections. Both Airbnb and Booking.com cater effectively to these preferences with their unique listings and local perspectives. Expedia, while strong in conventional offerings, must enhance its focus on unique experiences to remain competitive in this evolving market landscape. Each platform has its strengths and weaknesses, making them suitable for different traveler demographics and preferences within the broader travel ecosystem.
```

1. **Develop a Platform Focused on Experience-Driven Travel**: Create a travel platform that prioritizes immersive, adventure-filled experiences catering to Gen Z's desire for unique and culturally-rich encounters. Incorporate user-generated content to showcase real-life experiences and promote activities such as cooking classes, adventure sports, and local craft workshops. This platform should enable easy customization of itineraries based on user interests and local offerings, enhancing the hyperlocal travel experience.

2. **Leverage Social Media and Influencer Partnerships**: Engage in aggressive digital marketing strategies that utilize platforms favored by Gen Z, such as Instagram and TikTok. Partner with relevant influencers who can authentically showcase travel experiences tailored to Gen Z's tastes. Utilize short-form video content to capture attention, and promote travel contests that encourage user participation and sharing, thereby enhancing brand visibility and engagement.

3. **Emphasize Sustainability and Affordable Travel Options**: Integrate sustainable travel options within your product offerings, allowing for socially conscious travel choices that resonate with Gen Z values. Offer transparent pricing, loyalty programs, and flexible booking options to cater to their frugal nature while creating a sense of value. Highlight eco-friendly activities and accommodations in marketing materials to align with the growing trend of ethical travel among younger generations.

This strategic approach ensures that the travel platform aligns with the modern preferences of Gen Z, providing a competitive edge in effectively capturing this vital market segment.